# Vector Graphics with Python
## Example: A rotating clock

In this example, we draw a "rotating" clock using only Pillow's ``Image.putpixel()`` method, a simple line drawing algorithm, and a few matrix transformations. 

You should have an understanding of:

* Python
* how lines are defined by ``y=m*x+t``
* transformation of homogeneous vectors via matrices
* image filtering via convolution (optional, for the blur filter)

Here's what we want to achieve:

In [ ]:
from IPython.display import Video
Video('rotating_clock.mp4', embed=True)

## Drawing Lines
Let's start at the beginning: we need to draw lines on the screen. While Pillow offers functions for drawing lines, we'll implement a simple one ourselves.

In [ ]:
from PIL import Image
from PIL.ImageOps import scale

In [ ]:
i = Image.new("L", (200,200)) # "L" = grayscale image (one channel, 8 bit)

In [ ]:
# Let's see what i contains ... darkness!
i

Let's write a simple line drawing method and iteratively improve it. 
Our intuition: given two points on the screen, go from left to right (increase the x value) and set the pixel to white for which the line equation y = m*x + t is true

In [ ]:
def naive_line(img, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    m = (y2-y1) / (x2-x1) # slope of the line: delta_y / delta_x
    # y = m*x + t  =>  t = y - m * x
    t = y1 - m*x1 # as the first point has to be on the line, we can use it to calculate t
    for x in range(x1, x2+1): # we iterate until x2+1 because we want to also set the pixel at p2
        y = int(m*x + t)
        img.putpixel((x,y), 255)
    return img

In [ ]:
naive_line(i, (0,0), (199,199))

In [ ]:
naive_line(i, (0,99), (199,0))

In [ ]:
naive_line(i, (199,99), (0,99))

**Huh?!** Where's the last line? 

It turns out that iterating from x1=199 to x2+1=1 using Python's ``range()`` function does not work. 
The function expects the lower value as the first argument.
So, let's fix this issue: if x1 > x2, we'll just swap the points so that x1 is again lower than (or equal to) x2.

In [ ]:
def naive_line(img, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if x1 > x2: # new: let's swap p1 and p2 if p1 is right of p2
        x1, x2 = x2, x1
        y1, y2 = y2, y1
    m = (y2-y1) / (x2-x1)
    t = y1 - m*x1
    for x in range(x1, x2+1):
        y = int(m*x + t)
        img.putpixel((x,y), 255)
    return img

In [ ]:
naive_line(i, (199,99), (0,99))

Now it works. Great! Let's draw a final line to check everything works...

In [ ]:
naive_line(i, (0,0), (20,199))

Oh, why are we drawing a dotted line now? Lez's have a closer look

In [ ]:
scale(i, 5)

As we iterate along the x axis, we always set exactly one pixel in each column. This leaves those gaps. 
While we could extend our algorithm to set multiple pixels per column, there is an easier way: 
    
If the line is steeper than 45°, we simply iterate along the y axis instead of the x axis. 
And because we don't want too much duplicated code, we'll just swap x and y axis before the loop and then 
swap them back before drawing

In [ ]:
def naive_line(img, p1, p2, color=255):
    vertical = False  # new: we'll use this flag to remember along which axis we are drawing
    x1, y1 = p1
    x2, y2 = p2
    if abs(y2-y1) > abs(x2-x1):  # new: swap x and y axis if delta_y > delta_x, i.e., we have a steep line
        vertical = True
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
    m = (y2-y1) / (x2-x1)
    t = y1 - m*x1
    for x in range(x1, x2+1):
        y = int(m*x + t)
        if vertical:
            x, y = y, x # swap pixels
        img.putpixel((x,y), color)
    return img

In [ ]:
naive_line(i, (0,0), (20,199))

Yay, we've written a line drawing function that works for all cases ... let's handle the corner cases. 

What could these be?

In [ ]:
naive_line(i, (0,0), (0,0)) # lines of length zero

In [ ]:
naive_line(i, (0,100), (300,300)) # trying to set pixels outside our canvas

In [ ]:
def naive_line(img, p1, p2, color=128): # new: draw less bright
    vertical = False 
    w, h = img.size # new: remember the size of the image
    x1, y1 = p1
    x2, y2 = p2
    if abs(y2-y1) > abs(x2-x1):
        vertical = True
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
    if x2 == x1: # handle line of length 0
        return img
    m = (y2-y1) / (x2-x1)
    t = y1 - m*x1
    for x in range(x1, x2+1):
        y = int(m*x + t)
        if vertical:
            x, y = y, x
        if (x <= w+1 and y <= h-1): # new: only draw pixels within the canvas
            img.putpixel((x,y), color) 
    return img

In [ ]:
naive_line(i, (0,0), (0,0)) # lines of length zero

In [ ]:
naive_line(i, (0,100), (300,300)) # trying to set pixels outside our canvas

### Small trick: adding a new method to a class

In [ ]:
Image.Image.line = naive_line
i = Image.new("L",(100,100))
i.line((10,10), (30,70)) # -> naive_line(i, (10,10), (30,70))

This only works because 

- you can dynamically add methods to classes (``Image.Image.line = naive_line``)
- methods are not really different than normal functions - 
  Python just translates calls to ``obj.line(p1, p2)`` to ``line(obj, p1, p2)``
- the first parameter of `ǹaive_line()`` (``img``) expects an ``Image`` object - and the aforementioned mechanism
  results in a call of `ǹaive_line(i, p1, p2)`` - i.e., passes our image object to the function
  
We will use this hack in the following. Also, let's add a convenience function that draws a marker

In [ ]:
def within(img, x, y):
    w, h = img.size
    x, y = int(x), int(y)
    if x >= 0 and x < w and y >= 0 and y < h: 
        return True
    else:
        return False
        
def marker(img, p, size=3, color=255):
    if size < 1:
        return
    #img.putpixel(p, color)
    x, y = map(int, p)
    for i in range(-size+1, size):
        if within(img, x+i, y):
            img.putpixel((x+i,y), color) 
        if within(img, x, y+i):
            img.putpixel((x,y+i), color) 
    return img

Image.Image.marker = marker

In [ ]:
i.marker((50,50))

## Drawing Shapes

We'll just draw unfilled shapes for now because filled shapes are actually a little bit more difficult

In [ ]:
# a generator that translates a tuple (p1, p2, p3) into a series of tuples (p1, p2), (p2, p3), (p3,p1)
def pairwise_wrap(l):
    l = iter(l)
    first = next(l, None)
    prev = first
    while o := next(l, False):
        yield (prev, o)
        prev = o
    yield (prev, first)

In [ ]:
def draw_shape(img, shape, color=128, markers=True):
    for p1, p2 in pairwise_wrap(shape):
        naive_line(img, map(int, p1), map(int, p2), color)
        if markers:
            img.marker(p1)
    return img
        
Image.Image.draw_shape = draw_shape # also assign this function to the Image class

In [ ]:
i = Image.new("L",(100,100))
rect = ((10,10), (10,90), (90,90), (90,10))
i.draw_shape(rect)

## Transformations

In [ ]:
import numpy as np
from numpy import matrix as M
from math import sin, cos, pi

In [ ]:
A = M('1 0; 0 1')
B = M([[-1, 0], [1,0]])
A, B

In [ ]:
A * 3

In [ ]:
v = (2,3)

In [ ]:
v * A

In [ ]:
A * v

In [ ]:
A @ v

In [ ]:
v @ B, B @ v

## Matrix Transformations

Let's write a few functions that apply affine transformations to a point/vector.

Two small details: 

- We use homogenous coordinates (x,y,1) so that we can represent all transformations as matrix multiplications
- All functions can be called withouth providing a point/vector as a parameter. In this case, the respective transformation matrix is returned. We can use these later.

In [ ]:
def translate(tx, ty, p=None):
    T = M([[1, 0, tx],
           [0, 1, ty],
           [0, 0, 1]])
    if p is None:
        return T
    else:
        p = list(p)
        p.append(1)
        p = T @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        return((x,y))

In [ ]:
translate(97, 13, (3,-13))

In [ ]:
def rotate(angle, p=None):
    # degree to radian
    angle = angle / (180/pi)
    R = M([[cos(angle), -sin(angle), 0],
          [sin(angle), cos(angle), 0],
          [0, 0, 1]])
    if p is None:
        return R
    else:
        p = list(p)
        p.append(1)
        p = R @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        return((x,y))

In [ ]:
rotate(45, (2,2))

In [ ]:
def scale(sx, sy, p=None):
    S = M([[sx, 0, 0],
           [0, sy, 0],
           [0, 0, 1]])
    if p is None:
        return S
    else:
        p = list(p)
        p.append(1)
        p = S @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        return((x,y))

In [ ]:
scale(3, -0.3, (20, 5))

In [ ]:
rotate(180, [10,0])

In [ ]:
i = Image.new("L",(100,100))
for a in range(0, 360, 5):
    _ = i.marker(map(int, translate(50,50, rotate(a, [40,0]))))
display(i)

In [ ]:
i = Image.new("L",(100,100))
i.marker((10,0), 2)
i.marker(scale(4, 4, (10,0)), 4)
i.marker(translate(20,20, scale(4, 4, (10,0))), 6)

In [ ]:
A = translate(20,20)
B = rotate(45)
print(A @ B)

In [ ]:
C = A @ B
C @ (10,10, 1)

### Next steps

Draw a clock with hands and hour indicators.
Bonus: have it show the current time

In [ ]:
from datetime import datetime as dt

In [ ]:
#def draw_shape(img, shape, color=255):
#    for p1, p2 in pairwise_wrap(shape):
#        naive_line(img, map(int, p1), map(int, p2), color)

In [ ]:
def transform(shape, matrix):
    ret = []
    for point in shape:
        if len(point) == 2:
            point = list(point)
            point.append(1)
        elif len(point) != 3:
            raise ValueError("Points need to have a length of 2 or 3.")
        p = matrix @ point
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        ret.append((x,y))
    return ret

In [ ]:
i = Image.new("L",(100,100))
rect = ((10,10), (10,90), (90,90), (90,10))
rect2 = transform(rect, translate(50,50) @ rotate(45) @ scale(0.5, 0.5) @ translate(-50,-50))
i.draw_shape(rect2)

In [ ]:
def clock(warp = None, flt = None, color = 255, size = 400):
    # assuming width/height of 400
    hour_mark = ((0, 5), (45,5), (45,-5), (0,-5))
    hour_hand = ((-5,5), (5,5), (5, -120), (-5, -120))
    if warp is None:
        warp = M('1 0 0; 0 1 0; 0 0 1') # identity
    i = Image.new("L", (size, size))
    # marks
    for a in range(0, 360, 30):
        mat = warp @ translate(size/2,size/2) @ rotate(a) @ translate(size/3, 0) @ scale(size/400, size/400)
        mark = transform(hour_mark, mat)
        draw_shape(i, mark, int(color*0.7))
    # hand
    hour = dt.now().hour # 0-23
    hour_mat = warp @ translate(size/2,size/2) @ rotate(30 * hour) @ scale(size/400, size/400)
    hand = transform(hour_hand, hour_mat)
    draw_shape(i, hand, color)
    # minute
    minute = dt.now().minute # 0-59
    minute_mat = warp @ translate(size/2,size/2) @ rotate(6 * minute) @ scale(0.5, 1.3) @ scale(size/400, size/400)
    hand = transform(hour_hand, minute_mat)
    draw_shape(i, hand, color)
    # second
    second = dt.now().second # 0-59
    second_mat = warp @ translate(size/2,size/2) @ rotate(6 * second) @ scale(0.2, 1.5) @ scale(size/400, size/400)
    hand = transform(hour_hand, second_mat)
    draw_shape(i, hand, color)
    if flt is None:
        return i
    else:
        return flt(i)

In [ ]:
from IPython.display import DisplayHandle
from time import sleep
d = DisplayHandle()
d.display(clock())

In [ ]:
while True:
    d.update(clock())
    sleep(0.1)

## Filters!

In [ ]:
def convolution(kernel, image):
    w, h = image.size
    img_out = image.copy()
    div = sum(kernel[0]) + sum(kernel[1]) + sum(kernel[2])
    for x in range(1, w-1):
        for y in range(1, h-1):
            new_val = 0
            for i in range(3):
                for j in range(3):
                    new_val += int(image.getpixel((x-1+i, y-1+j)) *
                                    kernel[i][j] / (div+0.01))
            img_out.putpixel((x,y), new_val)
    return img_out

def blur(image):
    kernel = [[1,1,1],
              [1,1,1],
              [1,1,1]]
    return convolution(kernel, image)

In [ ]:
d = DisplayHandle()
d.display(clock(None, blur))
while True:
    d.update(clock(None, blur))
    sleep(1)

### Faster convolution

In [ ]:
from scipy.signal import convolve2d
def fast_blur(img):
    kernel = [[0,1,0],
              [1,1,1],
              [0,1,0]]
    blurred = convolve2d(np.array(img, dtype=np.uint16), np.array(kernel, dtype=np.uint16), mode = "same") / 5 
    #print(blurred)
    return Image.fromarray(np.asarray(blurred, dtype=np.uint8))

In [ ]:
fast_blur(i)

In [ ]:
d = DisplayHandle()
d.display(clock(None, fast_blur))
while True:
    d.update(clock(None, fast_blur))
    sleep(1)

## Some Animation!

In [ ]:
w = 0.5
change = -0.01
while True:
    w += change
    if w <= -0.95 or w >= 0.95:
        change = -change
    mat = translate(200,0) @ scale(w, 1.0) @ translate(-200,0)
    d.update(clock(mat))
    sleep(0.01)

In [ ]:
size = 400
a = 0
change = 0.01
while True:
    a += change
    if a >= 2*pi:
        a = 0
    color = int(50 + abs(sin(a)) * 200)
    mat = translate(size/2,0) @ scale(sin(a), 1.0) @ translate(-size/2,0)
    d.update(clock(mat, fast_blur, color, size))
    sleep(0.01)